# Python: คำนวณ สส ในสภา แบบ พรรคเล็กได้ สส.

<img src="Pic_data/Python_Logo.png" width = 400>


<img src="Pic_data/PY-01_Elephant.jpg" style="width: 400px;">

https://www.facebook.com/elephant.data.th/

## Objective
* เข้าใจระบบการคำนวณ สส โดยใช้ pandas dataframe manipulation

## Calculation Reference
* https://workpointnews.com/2019/04/02/%E0%B9%80%E0%B8%9B%E0%B8%B4%E0%B8%94%E0%B8%A7%E0%B8%B4%E0%B8%98%E0%B8%B5%E0%B8%84%E0%B8%B4%E0%B8%94-2-%E0%B8%AA%E0%B8%B9%E0%B8%95%E0%B8%A3%E0%B8%84%E0%B8%B3%E0%B8%99%E0%B8%A7%E0%B8%93-%E0%B8%AA/

## Data Source

* http://news.thaipbs.or.th/content/278820?fbclid=IwAR2JicWI_KErRVugqB11fNpJqejTbNDDfHDTrxEHiKyrtMm9qznA1eC2v2k

## 1. Load data

In [ ]:
import pandas as pd

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html

main_df  = pd.read_excel(io = 'Election_62_input.xlsx',
                         index_col = 0)
main_df

## 2. คำนวณ vote ทั้งหมด และ จำนวน vote ที่ต้องได้ต่อ สส party list 1 คน

In [ ]:
# จำนวน vote ท้ังหมด

vote_total_absolute = main_df['total_score'].sum()
vote_total_absolute 

In [ ]:
# จำนวน vote ท้ังหมด คิดเฉพาะ พรรคที่ส่ง party list

vote_total = main_df[main_df['join_party_list'] == 'Y']['total_score'].sum()
vote_total

In [ ]:
# define total rep (จำนวน สส ทั้งหมด)

rep_total  = 500
rep_total

In [ ]:
# จำนวน vote ที่ต้องมีเพื่อ ได้ สส party list 1 คน

vote_per_rep = vote_total / rep_total
vote_per_rep

## 3. คำนวณ สส พึงมี (เพดานที่แต่ละพรรคจะมีได้)

In [ ]:
# จำนวน สส พึงมี คือ vote / สส ที่ require สส 1 คน
# การคำนวณอีกแบบ จะใกล้เคียงค่านี้

main_df['rep_expect'] = main_df['total_score'] / vote_per_rep
main_df

## 4. คำนวณ สส party list ของแต่ละพรรค

In [ ]:
# จำนวน สส party list ที่มี คือ rep_expect - rep_district (สส พีงมี - สส เขต)

main_df['rep_party_list'] =  main_df['rep_expect'] - main_df['rep_district']
main_df

## 5. กรณี พรรค เพื่อไทย มี สส party list ติดลบ จะปรับเป็น 0

In [ ]:
# เลือก เฉพาะ พรรค ที่ สส party list ติดลบ ให้ = 0

main_df.loc[main_df['rep_party_list'] < 0,'rep_party_list'] = 0
main_df

## 6. คำนวณ สส Party List รวม


In [ ]:
# รวม สส ที่มี จาก สส party list ในตาราง รวมกับ พรรค ที่ไม่มีในตาราง
party_list_sum = main_df['rep_party_list'].sum()
party_list_sum 

## 7. จะพบว่า จำนวน สส มีเกิน 150 คน

#### เทียบบรรญัติไตรยางค์ใหม่ ให้ได้ สส party list รวมเป็น 150

โดยการ คูณ 150 / 175.5321

In [ ]:
# normalize จำนวน สส

main_df['rep_party_list_norm'] = main_df['rep_party_list'] * 150 / party_list_sum 
main_df

## 8. แจก สส รอบจำนวนเต็ม

In [ ]:
# คิดเฉพาะ จำนวนเต็มที่ได้ก่อน แล้วแจก สส party list ตามจำนวนเต็ม

main_df['rep_party_list_norm_int'] = main_df['rep_party_list_norm'].apply(int)
main_df

## 9. คำนวณ สส รวม รอบจำนวนเต็ม และ จำนวน สส ที่ต้องแจกเพิ่มในรอบเศษส่วน

In [ ]:
# ทำ summation รอบจำนวนเต็ม

total_rep_int = main_df['rep_party_list_norm_int'].sum()
total_rep_int

In [ ]:
# มี สส ที่ต้องกระจายเพิ่ม 150 - 129 = 21

remain_rep = 150 - total_rep_int
remain_rep

## 10. คำนวณเฉพาะ สส ที่เป็นเศษ หลังจากการปัดจำนวนเต็ม

In [ ]:
# คำนวณเฉพาะค่าเศษ (fraction)

main_df['rep_party_list_norm_fraction'] = main_df['rep_party_list_norm'] - main_df['rep_party_list_norm_int']
main_df

## 11. กระจายเพิ่มอีก 21 ที่นั่ง ตามเศษที่เยอะก่อน ผ่านการ sorting

In [ ]:
# ทำการ sort โดยใส่ค่า ranking ไป ตามการ sort descending จากมากไปน้อย (ค่ามากจะมี score ranking ต่ำกว่า)

main_df['fraction_rank'] = main_df['rep_party_list_norm_fraction'].rank(ascending = False)
main_df

## 12. เลือกพรรคที่มีโอกาสได้ สส เพิ่มจากรอบสัดส่วน  21 อันดับตาม fraction

In [ ]:
main_df[main_df['fraction_rank'] <= 21]

## 13. แจก สส เพิ่ม จากรอบสัดส่วน

In [ ]:
# initialize ค่า rep จากรอบสัดส่วนเป็น 0 ก่อน

main_df['rep_add_fraction'] = 0
main_df

In [ ]:
# แจก สส เพิ่มเติม จาก รอบสัดส่วน

main_df.loc[main_df['fraction_rank'] <= 21,'rep_add_fraction'] = 1

main_df
# main_df.loc['rep_add_fraction'][main_df['fraction_rank'] <= 21] = 1
# main_df

## 14. คำนวณ สส ทั้งหมดที่ได้ จาก partylist ทั้งหมด

In [ ]:
# รวม party list จากรอบจำนวนเต็ม และ รอบ สัดส่วน

main_df['Final_Party_List'] = main_df['rep_party_list_norm_int'] + main_df['rep_add_fraction'] 
main_df

## 15. คำนวณ สส ที่ได้ทั้งหมด

In [ ]:
## สส ทั้งหมด คือ rep district (สส เขต) + สส Party list final (party list หลังจากการคำนวณอันซับซ้อนด้านบน)

main_df['Final_rep'] = main_df['rep_district'] + main_df['Final_Party_List']
main_df

## 16. Select only useful column for exporting

In [ ]:
summary_df = main_df[['Party','total_score','rep_district','Final_Party_List','Final_rep']]
summary_df

## 17. Export to excel file

In [ ]:
summary_df.to_excel('Election_62_out.xlsx')